## Model testing on prepared data

In [ ]:
%reload_ext autoreload
%autoreload 2
import os
import sys
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import plotly.graph_objects as go

module_path = os.path.abspath(os.path.join('./../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src.multicovariate_models.gmm_functions import predict_groups_gmm, fit_gaussian_mixture_model
from src.multicovariate_models.GroupsModelWrapper import GroupsModelWrapper
from src.multicovariate_models.DataLoader import DataLoader

In [ ]:
%%time

main_df = pd.read_csv("simulated_sensor_data_8_cols.csv")
main_df = main_df.drop(columns=["origin_file"]).drop_duplicates()
"""main_df = main_df[['Sensor1', 'Sensor2', 'Sensor3', 'Sensor4', 'J-Apollo', 'J-RN2',
       'J-RN1', 'node_with_leak']].round(3).drop_duplicates()"""
display(main_df)

# DataLoader 
Here we prepare the necessary data: x_train, x_test, y_train, y_test to be used in our models later

In [ ]:
%%time

# prepare the data for all models
data_loader = DataLoader(main_df)
x_train, x_test, y_train, y_test, enc_node_dict = data_loader.get_random_data_split_by_node()
print(x_train, x_test, y_train, y_test)

In [ ]:
%%time

x_train_v2, x_test_v2, y_train_v2, y_test_v2, enc_node_dict_v2 = \
                        data_loader.get_sequential_subsample_data_split_by_leaks(subsample_size=0.15)
print(x_train_v2, x_test_v2, y_train_v2, y_test_v2)

# GMM model testing

In [ ]:
%%time
# sklearn GroupShuffleSplit and pandas.cut may come in handy for splitting data sometime

# GMM model testing
model_random_s = GroupsModelWrapper(x_train, x_test, y_train, y_test, enc_node_dict, model_type="GMM")
model_seq_s = GroupsModelWrapper(x_train_v2, x_test_v2, y_train_v2, y_test_v2, enc_node_dict_v2, 
                                 model_type="GMM")

In [ ]:
%%time

model_seq_s.train()

In [ ]:
%%time
model_seq_s.evaluate_model_on_node_basis()

In [ ]:
%%time

model_random_s.train()
model_random_s.evaluate_model_on_node_basis()

## SVM model testing

In [ ]:
%%time

# SVM model testing
svm_model_random_s = GroupsModelWrapper(x_train, x_test, y_train, y_test, enc_node_dict, model_type="SVM")
svm_model_seq_s = GroupsModelWrapper(x_train_v2, x_test_v2, y_train_v2, y_test_v2, enc_node_dict_v2, 
                                 model_type="SVM")

In [ ]:
%%time

svm_model_seq_s.train()
svm_model_seq_s.evaluate_model_on_node_basis()

In [ ]:
%%time

svm_model_random_s.train()
svm_model_random_s.evaluate_model_on_node_basis()